In [3]:
# convert json to label studio format

import json
json_file = '/home/pgajo/working/food/data/TASTEset/data/TASTEset_semicolon_formatted_en-it_unaligned_aligned_model=mdeberta-v3-base-xl-wa_recipe_aligner_5epochs_error_rate=0.0119.json'
with open(json_file) as f:
    data = json.load(f)

recipe_list = data['annotations']

new_dataset = []

for i, recipe in enumerate(recipe_list):
    entry = {}
    entry['text_en'] = recipe['text_en']
    entry['text_it'] = recipe['text_it']
    entry['entities_en'] = recipe['entities_en']
    entry['entities_it'] = recipe['entities_it']
    entry['id'] = i
    new_dataset.append(entry)    

new_json_file = json_file.replace('.json', '_label_studio.json')
with open(new_json_file, 'w') as f:
    json.dump(new_dataset, f)

In [2]:
import spacy
import pandas as pd
import json
from itertools import groupby

# Download spaCy models:
models = {
    'en_core_web_sm': spacy.load("en_core_web_sm"),
    'en_core_web_lg': spacy.load("en_core_web_lg")
}

# This function converts spaCy docs to the list of named entity spans in Label Studio compatible JSON format:
def doc_to_spans(doc):
    tokens = [(tok.text, tok.idx, tok.ent_type_) for tok in doc]
    results = []
    entities = set()
    for entity, group in groupby(tokens, key=lambda t: t[-1]):
        if not entity:
            continue
        group = list(group)
        _, start, _ = group[0]
        word, last, _ = group[-1]
        text = ' '.join(item[0] for item in group)
        end = last + len(word)
        results.append({
            'from_name': 'label',
            'to_name': 'text',
            'type': 'labels',
            'value': {
                'start': start,
                'end': end,
                'text': text,
                'labels': [entity]
            }
        })
        entities.add(entity)

    return results, entities

# Now load the dataset and include only lines containing "Easter ":
df = pd.read_csv('/home/pgajo/working/food/data/TASTEset/src/lines_clean.csv', encoding='utf-8')
df = df[df['line_text'].str.contains("Easter ", na=False)]
print(df.head())
texts = df['line_text']

# Prepare Label Studio tasks in import JSON format with the model predictions:
entities = set()
tasks = []
for text in texts:
    predictions = []
    for model_name, nlp in models.items():
        doc = nlp(text)
        spans, ents = doc_to_spans(doc)
        entities |= ents
        predictions.append({'model_version': model_name, 'result': spans})
    tasks.append({
        'data': {'text': text},
        'predictions': predictions
    })

# Save Label Studio tasks.json
print(f'Save {len(tasks)} tasks to "tasks.json"')
with open('/home/pgajo/working/food/data/TASTEset/src/lines_tasks.json', mode='w') as f:
    json.dump(tasks, f, indent=2)
    
# Save class labels as a txt file
print('Named entities are saved to "named_entities.txt"')
with open('/home/pgajo/working/food/data/TASTEset/src/lines_named_entities.txt', mode='w') as f:
    f.write('\n'.join(sorted(entities)))

/home/pgajo/working/food/food-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                         act_name  episode_id  \
35                        Act One           1   
36                        Act One           1   
39                        Act One           1   
2081   Act Two. Dad In The Closet          10   
10691  Act One. Die In Six Months          50   

                                               line_text        speaker  \
35     We will not attempt to answer these questions,...      Ira Glass   
36     So I entered Jerusalem on Easter with a simple...    Kevin Kelly   
39     I slept there until early morning, when the ac...    Kevin Kelly   
2081   Shall we take the Easter Bunny and Santa Claus...  Jerry Walters   
10691  But all that changed. When he was 27, he came ...      Ira Glass   

      speaker_class    timestamp  
35             host  00:07:39.45  
36             host  00:08:05.95  
39             host  00:10:27.35  
2081        subject  00:31:58.77  
10691          host  00:05:11.59  
Save 34 tasks to "tasks.json"
Named entities